In [1]:
import pandas as pd

In [2]:
### This file can be obtained by running the scripts PM_AD/Code/DNAm_bulk_transcriptome/Triplet/1_Triplet_analysis.R and PM_AD/Code/DNAm_bulk_transcriptome/Triplet/2_TFBS_triplet.R sequentially

In [3]:
Proximal= pd.read_csv("/data_hdd1/yang/submission_data/Triplet_output/Proximal_triplet_TFBS.txt",index_col=0)
Distal = pd.read_csv("/data_hdd1/yang/submission_data/Triplet_output/Distal_triplet_TFBS.txt",index_col=0)

In [4]:
triplet = pd.concat([Proximal, Distal] ,axis =0)

In [5]:
def Triplet_filtering(tmp1):
    # Separate data based on TF role
    Activator = tmp1[tmp1['TF.role'] == 'Activator']
    Repressor = tmp1[tmp1['TF.role'] == 'Repressor']
    # Dual role is excluded as we are focusing on cases where DNA methylation attenuates TF activity.
    
    ##########################################
    # DNA methylation is low, and expression in the low methylation group increases due to Activator
    low_meth_active = Activator[Activator.DNAm_low_RLM_target_vs_TF_estimate > 0]  # Regression direction
    low_meth_active = low_meth_active[low_meth_active.DNAm_low_RLM_target_vs_TF_pvalue < 0.05]  # Statistical significance
    low_meth_active = low_meth_active[low_meth_active.DNAm_low_RLM_target_vs_TF_estimate > low_meth_active.DNAm_high_RLM_target_vs_TF_estimate]
    low_meth_active = low_meth_active[low_meth_active.RLM_DNAmGroup_estimate < 0]  # Low methylation group has higher expression
    
    ##########################################
    # DNA methylation is low, and expression in the low methylation group decreases due to Repressor
    low_meth_repress = Repressor[Repressor.DNAm_low_RLM_target_vs_TF_estimate < 0]
    low_meth_repress = low_meth_repress[low_meth_repress.DNAm_low_RLM_target_vs_TF_pvalue < 0.05]
    low_meth_repress = low_meth_repress[low_meth_repress.DNAm_low_RLM_target_vs_TF_estimate < low_meth_repress.DNAm_high_RLM_target_vs_TF_estimate]
    low_meth_repress = low_meth_repress[low_meth_repress.RLM_DNAmGroup_estimate > 0]
    
    # Concatenate all filtered data
    data = pd.concat([low_meth_active, low_meth_repress]).reset_index(drop=True)
    
    return data



In [6]:
triplet = Triplet_filtering(triplet)

In [7]:
triplet = triplet[triplet['TF_binding_pvalue'] < 0.05].reset_index(drop=True)

In [8]:
triplet[triplet.target_symbol =='FCER1G']

,X,regionID,probeID,target_symbol,target,TF_symbol,TF,distance_region_target_tss,target_region,met.IQR,...,X..of.target.genes.not.expressed.in.DNAm_low.and.DNAm_high,DNAm_low_RLM_target_vs_TF_pvalue,DNAm_low_RLM_target_vs_TF_estimate,DNAm_high_RLM_target_vs_TF_pvalue,DNAm_high_RLM_target_vs_TF_estimate,DNAm.effect,TF.role,TF_binding_pvalue,GFF3_info,strand
0,73,chr1:161217049-161217050,cg20806175,FCER1G,ENSG00000158869,RELB,ENSG00000104856,1814,chr1:161215234-161220699,0,...,0 %,0.027608,0.250305,0.506529,-0.107926,Attenuating,Activator,0.0468,TF=RELB;class=Rel homology region (RHR) factor...,-
4,1179,chr1:161201593-161201594,cg05656486,FCER1G,ENSG00000158869,RUNX1,ENSG00000159216,-13639,chr1:161215234-161220699,0,...,0 %,0.000036,0.313558,0.872475,0.015555,Attenuating,Activator,0.0263,TF=RUNX1;class=Runt domain factors;sequence=GA...,+
5,1203,chr1:161201593-161201594,cg05656486,FCER1G,ENSG00000158869,NFE2L2,ENSG00000116044,-13639,chr1:161215234-161220699,0,...,0 %,0.001235,0.301280,0.073274,-0.272060,Attenuating,Activator,0.0216,TF=NFE2L2;class=Basic leucine zipper factors (...,+


In [10]:
triplet

,X,regionID,probeID,target_symbol,target,TF_symbol,TF,distance_region_target_tss,target_region,met.IQR,...,X..of.target.genes.not.expressed.in.DNAm_low.and.DNAm_high,DNAm_low_RLM_target_vs_TF_pvalue,DNAm_low_RLM_target_vs_TF_estimate,DNAm_high_RLM_target_vs_TF_pvalue,DNAm_high_RLM_target_vs_TF_estimate,DNAm.effect,TF.role,TF_binding_pvalue,GFF3_info,strand
0,73,chr1:161217049-161217050,cg20806175,FCER1G,ENSG00000158869,RELB,ENSG00000104856,1814,chr1:161215234-161220699,0,...,0 %,0.027608,0.250305,0.506529,-0.107926,Attenuating,Activator,0.0468,TF=RELB;class=Rel homology region (RHR) factor...,-
1,172,chr2:75711234-75711235,cg01044662,GCFC2,ENSG00000005436,TP63,ENSG00000073282,-248,chr2:75652000-75710985,0,...,0 %,0.039438,0.102720,0.214738,-0.062403,Attenuating,Activator,0.0380,TF=TP63;class=p53 domain factors;sequence=AAAG...,-
2,1209,chr20:53608210-53608211,cg08935725,ZNF217,ENSG00000171940,AR,ENSG00000169083,1695,chr20:53567065-53609907,0,...,0 %,0.010055,0.067944,0.504754,-0.035509,Attenuating,Activator,0.0294,TF=AR;class=Nuclear receptors with C4 zinc fin...,-
3,446,chr1:44660400-44660401,cg15564347,RPS8,ENSG00000142937,BHLHE22,ENSG00000180828,-114849,chr1:44775251-44778779,0,...,0 %,0.004132,0.157890,0.460818,-0.041967,Attenuating,Activator,0.0151,TF=BHLHE22;class=Basic helix-loop-helix factor...,-
4,1179,chr1:161201593-161201594,cg05656486,FCER1G,ENSG00000158869,RUNX1,ENSG00000159216,-13639,chr1:161215234-161220699,0,...,0 %,0.000036,0.313558,0.872475,0.015555,Attenuating,Activator,0.0263,TF=RUNX1;class=Runt domain factors;sequence=GA...,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,11386,chr17:56681573-56681574,cg03560743,TRIM25,ENSG00000121060,HNF4A,ENSG00000101076,232505,chr17:56836387-56914080,0,...,0 %,0.002354,-0.599033,0.127127,-0.140670,Attenuating,Repressor,0.0082,TF=HNF4A;class=Nuclear receptors with C4 zinc ...,+
60,11398,chr17:56681573-56681574,cg03560743,TRIM25,ENSG00000121060,SNAI2,ENSG00000019549,232505,chr17:56836387-56914080,0,...,0 %,0.000424,-0.299668,0.932191,0.009739,Attenuating,Repressor,0.0152,TF=SNAI2;class=C2H2 zinc finger factors;sequen...,-
61,11410,chr17:56681573-56681574,cg03560743,TRIM25,ENSG00000121060,BHLHE22,ENSG00000180828,232505,chr17:56836387-56914080,0,...,0 %,0.003878,-0.358103,0.250859,-0.096617,Attenuating,Repressor,0.0085,TF=BHLHE22;class=Basic helix-loop-helix factor...,+
62,14345,chr21:33079131-33079132,cg07363526,LINC01548,ENSG00000229086,MYCN,ENSG00000134323,91516,chr21:33164809-33170649,0,...,0 %,0.008509,-0.131217,0.944005,0.002463,Attenuating,Repressor,0.0021,TF=MYCN;class=Basic helix-loop-helix factors (...,+


In [11]:
# Filter with epigenetic data
# 1. Check if CpG_Target genes are within the same TAD
# 2. Check if CpG methylation overlaps with the peaks of H3K27ac, indicating enhancer regions 

In [12]:
### create bed format.

In [13]:
CpG = [
    float(triplet.target_region.str.split(":|-")[i][1]) + 
    (triplet.distance_region_target_tss[i] + 1 if triplet.distance_region_target_tss[i] > 0 else triplet.distance_region_target_tss[i] - 2)
    for i in range(len(triplet))
]


In [14]:
Target_TSS = [float(triplet.target_region.str.split(":|-")[i][1]) for i in range(len(triplet))]
chrom = [triplet.regionID.str.split(":")[i][0] for i in range(len(triplet))]
label = triplet['probeID'] + "_" + triplet['target_symbol']
CpG_Target_bed = pd.DataFrame(chrom)


In [15]:
CpG_Target_bed[1] = 0
CpG_Target_bed[2] = 0
CpG_Target_bed[3] = 0

In [16]:
for i in range(len(CpG_Target_bed)):
    if triplet.loc[i, 'distance_region_target_tss'] > 0:
        CpG_Target_bed.loc[i, 1] = Target_TSS[i]
        CpG_Target_bed.loc[i, 2] = CpG[i]
    else:
        CpG_Target_bed.loc[i, 1] = CpG[i]
        CpG_Target_bed.loc[i, 2] = Target_TSS[i]

In [17]:
CpG_Target_bed[3] = label

In [18]:
CpG_Target_bed = CpG_Target_bed.drop_duplicates()

In [19]:
CpG_Target_bed

,0,1,2,3
0,chr1,161215234,161217049,cg20806175_FCER1G
1,chr2,75651750,75652000,cg01044662_GCFC2
2,chr20,53567065,53568761,cg08935725_ZNF217
3,chr1,44660400,44775251,cg15564347_RPS8
4,chr1,161201593,161215234,cg05656486_FCER1G
6,chr2,61888724,62265273,cg12753957_COMMD1
7,chr2,102227650,102432289,cg26788216_MIR4772
9,chr2,102311502,102514939,cg02937179_IL1RL1
10,chr3,48117389,48290722,cg02894485_NME6
11,chr5,104910928,105099473,cg12829002_RAB9BP1


In [20]:
CpG_Target_bed.to_csv(
    "/home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/triplet.bed", 
    sep='\t', 
    index=False,  # 
    header=False  # 
)

### 1. Check if CpG_Target genes are within the same TAD

In [124]:
### bedtools code

In [21]:
''' bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/triplet.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_Hi_C/10kb_to_100kb_domains.bed -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet.bed
'''

' bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/triplet.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_Hi_C/10kb_to_100kb_domains.bed -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet.bed\n'

In [22]:
TAD_res = pd.read_csv("/home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet.bed", sep ='\t', header=None)

In [24]:
TAD_res[3]

0        cg20806175_FCER1G
1         cg01044662_GCFC2
2        cg08935725_ZNF217
3        cg05656486_FCER1G
4        cg01166253_MIR596
5         cg14378925_VPS51
6     cg00695387_LINC01488
7     cg02104112_LINC01488
8          cg06338928_HMBS
9         cg03913806_KRT35
10      cg22557409_TMPRSS9
11        cg01643250_ILVBL
12      cg06765124_RANGAP1
13        cg01461254_NRIP2
14        cg05194726_NRIP2
15        cg19612216_PSMD4
16       cg20784014_HHIPL2
17       cg12593397_IL1RL1
18         cg07524919_TNXB
19         cg17406422_CUBN
20    cg14542395_LINC00595
21       cg22160073_OR51Q1
22        cg09005886_PTPRO
23       cg04806741_RASAL1
24      cg10481691_ZNF280B
Name: 3, dtype: object

In [25]:
triplet['cg_target'] = triplet['probeID'] + "_" + triplet['target_symbol']

In [26]:
target_names = TAD_res[3].tolist()
matching_triplet = triplet[triplet['cg_target'].isin(target_names)]

In [27]:
matching_triplet['cg_target'].drop_duplicates()

0        cg20806175_FCER1G
1         cg01044662_GCFC2
2        cg08935725_ZNF217
4        cg05656486_FCER1G
13       cg01166253_MIR596
16        cg14378925_VPS51
18    cg00695387_LINC01488
19    cg02104112_LINC01488
20         cg06338928_HMBS
23        cg03913806_KRT35
25      cg22557409_TMPRSS9
31        cg01643250_ILVBL
33      cg06765124_RANGAP1
36        cg01461254_NRIP2
37        cg05194726_NRIP2
38        cg19612216_PSMD4
39       cg20784014_HHIPL2
40       cg12593397_IL1RL1
46         cg07524919_TNXB
47         cg17406422_CUBN
48    cg14542395_LINC00595
49       cg22160073_OR51Q1
53        cg09005886_PTPRO
56       cg04806741_RASAL1
63      cg10481691_ZNF280B
Name: cg_target, dtype: object

### 2. Check if CpG methylation overlaps with the active marker peaks of H3K27ac indicating enhancer regions (regulatory regions)

In [28]:
TAD_res

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr1,161215234,161217049,cg20806175_FCER1G,chr1,161180000,161440000,ID_0.01_411,-0.227053,.,161180000,161440000,"31,120,180"
1,chr2,75651750,75652000,cg01044662_GCFC2,chr2,75430000,75730000,ID_0.01_852,-0.203939,.,75430000,75730000,"51,160,44"
2,chr20,53567065,53568761,cg08935725_ZNF217,chr20,53550000,53960000,ID_0.01_6867,-0.276165,.,53550000,53960000,"31,120,180"
3,chr1,161201593,161215234,cg05656486_FCER1G,chr1,161180000,161440000,ID_0.01_411,-0.227053,.,161180000,161440000,"31,120,180"
4,chr8,1814011,1817231,cg01166253_MIR596,chr8,1690000,1860000,ID_0.01_3453,-0.284437,.,1690000,1860000,"31,120,180"
5,chr11,64868256,65089324,cg14378925_VPS51,chr11,64770000,65120000,ID_0.01_4655,0.304891,.,64770000,65120000,"31,120,180"
6,chr11,69471465,69481662,cg00695387_LINC01488,chr11,69090000,69680000,ID_0.01_4673,0.407923,.,69090000,69680000,"31,120,180"
7,chr11,69481662,69603303,cg02104112_LINC01488,chr11,69090000,69680000,ID_0.01_4673,0.407923,.,69090000,69680000,"31,120,180"
8,chr11,119084866,119317954,cg06338928_HMBS,chr11,119040000,119340000,ID_0.01_4789,0.193382,.,119040000,119340000,"31,120,180"
9,chr17,41459491,41476689,cg03913806_KRT35,chr17,41320000,41690000,ID_0.01_6203,0.376822,.,41320000,41690000,"31,120,180"


In [29]:
### Create CpG bed format.

In [30]:
cgID = [TAD_res[3].str.split("_")[i][0] for i in range(len(TAD_res))]
cgID = list(set(cgID))

In [31]:
CpG_region = triplet.loc[:,['probeID','regionID']].drop_duplicates()
CpG_region2 = CpG_region.set_index("probeID").loc[cgID].reset_index()

In [32]:
CpG_region2[['chr', 'start_end']] = CpG_region2['regionID'].str.split(':', expand=True)
CpG_region2[['start', 'end']] = CpG_region2['start_end'].str.split('-', expand=True)


bed_df = CpG_region2[['chr', 'start', 'end', 'probeID']]


bed_df['start'] = bed_df['start'].astype(int)   
bed_df['end'] = bed_df['end'].astype(int)         

/tmp/ipykernel_1906080/4121452868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bed_df['start'] = bed_df['start'].astype(int)
/tmp/ipykernel_1906080/4121452868.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bed_df['end'] = bed_df['end'].astype(int)


In [33]:
bed_df

,chr,start,end,probeID
0,chr1,151370423,151370424,cg19612216
1,chr12,113204994,113204995,cg04806741
2,chr20,53608210,53608211,cg08935725
3,chr11,119317954,119317955,cg06338928
4,chr11,69471465,69471466,cg00695387
5,chr17,41498337,41498338,cg03913806
6,chr2,102340820,102340821,cg12593397
7,chr1,161201593,161201594,cg05656486
8,chr12,2835314,2835315,cg05194726
9,chr11,5632175,5632176,cg22160073


In [34]:
bed_df.to_csv(
    "/home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet_CpG.bed", 
    sep='\t', 
    index=False,  
    header=False  
)

In [35]:
'''bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet_CpG.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_H3K27ac/H3K27ac_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_triplet.bed'''


'bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet_CpG.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_H3K27ac/H3K27ac_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_triplet.bed'

In [36]:
TAD_H3k27ac_res = pd.read_csv("/home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_triplet.bed", sep ='\t', header=None)
TAD_H3k27ac_res

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,chr12,113204994,113204995,cg04806741,chr12,113204050,113205099,NA_peak_34120,73,.,2.22282,7.30237,5.28060,697
1,chr20,53608210,53608211,cg08935725,chr20,53607719,53609306,NA_peak_82459,520,.,3.74060,52.03980,48.91010,660
2,chr1,161201593,161201594,cg05656486,chr1,161201051,161203891,NA_peak_9016,614,.,3.54147,61.47560,58.16050,986
3,chr12,2835314,2835315,cg05194726,chr12,2835203,2835487,NA_peak_28473,46,.,2.08845,4.60332,2.75791,177
4,chr2,75711234,75711235,cg01044662,chr2,75710131,75711408,NA_peak_72233,986,.,6.83186,98.68980,94.52120,668
5,chr1,161217049,161217050,cg20806175,chr1,161216567,161217091,NA_peak_9020,205,.,3.37952,20.57050,18.08420,201
6,chr22,41223722,41223723,cg06765124,chr22,41222769,41224348,NA_peak_87161,611,.,5.02611,61.14570,57.83730,301


In [37]:
target_names = TAD_H3k27ac_res[3].tolist()
matching_triplet2 = matching_triplet[matching_triplet['probeID'].isin(target_names)]

In [38]:
matching_triplet2['cg_target'].unique()

array(['cg20806175_FCER1G', 'cg01044662_GCFC2', 'cg08935725_ZNF217',
       'cg05656486_FCER1G', 'cg06765124_RANGAP1', 'cg05194726_NRIP2',
       'cg04806741_RASAL1'], dtype=object)

In [39]:
matching_triplet2

,X,regionID,probeID,target_symbol,target,TF_symbol,TF,distance_region_target_tss,target_region,met.IQR,...,DNAm_low_RLM_target_vs_TF_pvalue,DNAm_low_RLM_target_vs_TF_estimate,DNAm_high_RLM_target_vs_TF_pvalue,DNAm_high_RLM_target_vs_TF_estimate,DNAm.effect,TF.role,TF_binding_pvalue,GFF3_info,strand,cg_target
0,73,chr1:161217049-161217050,cg20806175,FCER1G,ENSG00000158869,RELB,ENSG00000104856,1814,chr1:161215234-161220699,0,...,0.027608,0.250305,0.506529,-0.107926,Attenuating,Activator,0.0468,TF=RELB;class=Rel homology region (RHR) factor...,-,cg20806175_FCER1G
1,172,chr2:75711234-75711235,cg01044662,GCFC2,ENSG00000005436,TP63,ENSG00000073282,-248,chr2:75652000-75710985,0,...,0.039438,0.102720,0.214738,-0.062403,Attenuating,Activator,0.0380,TF=TP63;class=p53 domain factors;sequence=AAAG...,-,cg01044662_GCFC2
2,1209,chr20:53608210-53608211,cg08935725,ZNF217,ENSG00000171940,AR,ENSG00000169083,1695,chr20:53567065-53609907,0,...,0.010055,0.067944,0.504754,-0.035509,Attenuating,Activator,0.0294,TF=AR;class=Nuclear receptors with C4 zinc fin...,-,cg08935725_ZNF217
4,1179,chr1:161201593-161201594,cg05656486,FCER1G,ENSG00000158869,RUNX1,ENSG00000159216,-13639,chr1:161215234-161220699,0,...,0.000036,0.313558,0.872475,0.015555,Attenuating,Activator,0.0263,TF=RUNX1;class=Runt domain factors;sequence=GA...,+,cg05656486_FCER1G
5,1203,chr1:161201593-161201594,cg05656486,FCER1G,ENSG00000158869,NFE2L2,ENSG00000116044,-13639,chr1:161215234-161220699,0,...,0.001235,0.301280,0.073274,-0.272060,Attenuating,Activator,0.0216,TF=NFE2L2;class=Basic leucine zipper factors (...,+,cg05656486_FCER1G
33,15030,chr22:41223722-41223723,cg06765124,RANGAP1,ENSG00000100401,THRB,ENSG00000151090,62527,chr22:41245611-41286251,0,...,0.014050,0.056902,0.954302,0.000770,Attenuating,Activator,0.0004,TF=THRB;class=Nuclear receptors with C4 zinc f...,+,cg06765124_RANGAP1
34,15044,chr22:41223722-41223723,cg06765124,RANGAP1,ENSG00000100401,NR2C2,ENSG00000177463,62527,chr22:41245611-41286251,0,...,0.000097,0.067973,0.878249,-0.002638,Attenuating,Activator,0.0426,TF=NR2C2;class=Nuclear receptors with C4 zinc ...,-,cg06765124_RANGAP1
35,15052,chr22:41223722-41223723,cg06765124,RANGAP1,ENSG00000100401,CEBPA,ENSG00000245848,62527,chr22:41245611-41286251,0,...,0.018961,0.161685,0.914706,0.003490,Attenuating,Activator,0.0038,TF=CEBPA;class=Basic leucine zipper factors (b...,-,cg06765124_RANGAP1
37,691,chr12:2835314-2835315,cg05194726,NRIP2,ENSG00000053702,MXI1,ENSG00000119950,228,chr12:2825348-2835544,0,...,0.015092,-0.012542,0.071129,0.053752,Attenuating,Repressor,0.0052,TF=MXI1;class=Basic helix-loop-helix factors (...,+,cg05194726_NRIP2
56,8566,chr12:113204994-113204995,cg04806741,RASAL1,ENSG00000111344,NFIC,ENSG00000141905,-68754,chr12:113098819-113136239,0,...,0.000006,-0.045545,0.080516,0.050699,Attenuating,Repressor,0.0327,TF=NFIC;class=SMAD/NF-1 DNA-binding domain fac...,-,cg04806741_RASAL1


In [40]:
matching_triplet2.to_excel("/data_hdd1/yang/submission_data/Triplet_output/Triplet_results.xlsx")

### H3k4me1

In [123]:
'''bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet_CpG.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_H3K4me1/H3K4me1_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_H3K4me1_triplet.bed'''


'bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_triplet.bed -b /home/yang/PM_AD/Submission/Code/Epigenetic_data/Data/Placenta_H3K4me1/H3K4me1_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_H3K4me1_triplet.bed'

### H3k4me3

In [124]:
'''bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_triplet_CpG.bed -b /data_hdd1/yang/Histone/H3K4me3/NA_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_H3K4me3_triplet.bed'''


'bedtools intersect -a /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_triplet.bed -b /data_hdd1/yang/Histone/H3K4me3/NA_peaks.narrowPeak -f 1.0 -wa -wb > /home/yang/PM_AD/Submission/Code/Epigenetic_data/Res/TAD_H3K27ac_H3K4me3_triplet.bed'